In [76]:
import pandas as pd

In [97]:
bike_realtime_df = pd.read_csv('./bike_realtime0416-0519.csv')
mrt_history_df = pd.read_csv('./mrt_history_groupby03-04.csv')
time_table_df = pd.read_csv('./time_table.csv')
youbike_mrt_distance_df = pd.read_csv('./youbike_mrt_distance.csv')
bike_station_df = pd.read_csv('./bike_station.csv')
mrt_station_info_df = pd.read_csv('./mrt_station_info.csv')
bike_realtime_df.drop(columns="create_time",inplace=True)

In [98]:
bike_realtime_df["source_time"] = pd.to_datetime(bike_realtime_df["source_time"]) 
bike_realtime_df["month"] = bike_realtime_df["source_time"].dt.month
bike_realtime_df["date"] = bike_realtime_df["source_time"].dt.date
bike_realtime_df["hour"] = bike_realtime_df["source_time"].dt.hour
bike_realtime_df["mins"] = bike_realtime_df["source_time"].dt.minute

In [99]:
bike_realtime_df["date"] = pd.to_datetime(bike_realtime_df["date"])
time_table_df["date"] = pd.to_datetime(time_table_df["date"])
mrt_history_df["date"] = pd.to_datetime(mrt_history_df["date"])

In [100]:
start_date = '2024-04-16'
end_date = '2024-04-30'

mrt_history_df = mrt_history_df[(mrt_history_df["date"] >= start_date) & (mrt_history_df["date"] <= end_date)]
bike_realtime_df = bike_realtime_df[(bike_realtime_df["date"] >= start_date) & (bike_realtime_df["date"] <= end_date)]

In [101]:
# bike_realtime_df = bike_realtime_df.loc[(bike_realtime_df["month"]==4)&((bike_realtime_df["hour"]>5) |(bike_realtime_df["hour"]==0)),].reset_index(drop=True)

In [102]:
mrt_history_df["month"] = mrt_history_df["date"].dt.month
mrt_history_df = mrt_history_df.loc[(mrt_history_df["month"]==4)&((mrt_history_df["hour"]>5) |(mrt_history_df["hour"]==0)),].reset_index(drop=True)

In [103]:
aval_bike_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_bike"].mean()).reset_index(drop=False)
aval_bike_mean.rename(columns={"aval_bike":"aval_bike_mean"},inplace=True)
aval_space_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_space"].mean()).reset_index(drop=False)
aval_space_mean.rename(columns={"aval_space":"aval_space_mean"},inplace=True)
df_bike_hour_grby = aval_bike_mean.merge(aval_space_mean,
                     how="left",
                     left_on=["bike_station_id","date","hour"],
                     right_on=["bike_station_id","date","hour"])

In [104]:
df  = df_bike_hour_grby.merge(bike_station_df.loc[:,["bike_station_id","station_name","total_space","lat","lng"]],
                              how="left",
                              on="bike_station_id")

In [105]:
time_table_df

,date,year,month,day,day_of_week,week_of_year,month_name,day_of_week_name
0,2023-01-01,2023,1,1,1,1,January,Sunday
1,2023-01-02,2023,1,2,2,1,January,Monday
2,2023-01-03,2023,1,3,3,1,January,Tuesday
3,2023-01-04,2023,1,4,4,1,January,Wednesday
4,2023-01-05,2023,1,5,5,1,January,Thursday
...,...,...,...,...,...,...,...,...
726,2024-12-27,2024,12,27,6,51,December,Friday
727,2024-12-28,2024,12,28,7,51,December,Saturday
728,2024-12-29,2024,12,29,1,52,December,Sunday
729,2024-12-30,2024,12,30,2,52,December,Monday


In [106]:
df = df.merge(time_table_df.loc[:,["date","day_of_week","day_of_week_name"]],
            on="date",
            how="left")


In [107]:
idx = youbike_mrt_distance_df.groupby("bike_station_id")["distance"].idxmin()
youbike_min_distance_table = youbike_mrt_distance_df.loc[idx,].reset_index(drop=True)

In [108]:
mrt_station_info_df.rename(columns={"station_name":"mrt_station_name"},inplace=True)

In [109]:
mrt_station_table = youbike_min_distance_table.merge(mrt_station_info_df.loc[:,["mrt_station_id","mrt_station_name"]],
                                 on="mrt_station_id",
                                 how="left")

In [110]:
df = df.merge(mrt_station_table,on="bike_station_id",how="left")
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,station_name,total_space,lat,lng,day_of_week,day_of_week_name,mrt_station_id,distance,mrt_station_name
0,500101001,2024-04-16,22,7.833333,20.166667,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,3,Tuesday,BR08,0.015941,科技大樓
1,500101001,2024-04-16,23,23.000000,5.000000,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,3,Tuesday,BR08,0.015941,科技大樓
2,500101001,2024-04-17,0,20.500000,7.500000,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓
3,500101001,2024-04-17,1,16.666667,11.333333,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓
4,500101001,2024-04-17,2,18.000000,10.000000,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438546,500119091,2024-04-29,16,11.500000,6.500000,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓
438547,500119091,2024-04-29,17,9.000000,9.000000,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓
438548,500119091,2024-04-29,18,0.500000,17.500000,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓
438549,500119091,2024-04-29,19,6.666667,11.333333,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓


In [111]:
df.dtypes

bike_station_id              int64
date                datetime64[ns]
hour                         int32
aval_bike_mean             float64
aval_space_mean            float64
station_name                object
total_space                  int64
lat                        float64
lng                        float64
day_of_week                  int64
day_of_week_name            object
mrt_station_id              object
distance                   float64
mrt_station_name            object
dtype: object

In [112]:
mrt_history_df

,date,hour,mrt_station,enter_num,exit_num,month
0,2024-04-16,0,龍山寺,127,142.0,4
1,2024-04-16,0,麟光,10,30.0,4
2,2024-04-16,0,頭前庄,15,35.0,4
3,2024-04-16,0,頂溪,71,230.0,4
4,2024-04-16,0,頂埔,8,104.0,4
...,...,...,...,...,...,...
33625,2024-04-30,23,三重國小,116,423.0,4
33626,2024-04-30,23,三重,87,183.0,4
33627,2024-04-30,23,三民高中,89,453.0,4
33628,2024-04-30,23,三和國中,99,356.0,4


In [113]:
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,station_name,total_space,lat,lng,day_of_week,day_of_week_name,mrt_station_id,distance,mrt_station_name
0,500101001,2024-04-16,22,7.833333,20.166667,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,3,Tuesday,BR08,0.015941,科技大樓
1,500101001,2024-04-16,23,23.000000,5.000000,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,3,Tuesday,BR08,0.015941,科技大樓
2,500101001,2024-04-17,0,20.500000,7.500000,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓
3,500101001,2024-04-17,1,16.666667,11.333333,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓
4,500101001,2024-04-17,2,18.000000,10.000000,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438546,500119091,2024-04-29,16,11.500000,6.500000,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓
438547,500119091,2024-04-29,17,9.000000,9.000000,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓
438548,500119091,2024-04-29,18,0.500000,17.500000,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓
438549,500119091,2024-04-29,19,6.666667,11.333333,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓


In [114]:
df = df.merge(mrt_history_df,left_on=["mrt_station_name","date","hour"],right_on=["mrt_station","date","hour"],how="left")

In [115]:
df["bike_rate"] = df["aval_bike_mean"]/df["total_space"]

In [116]:
df["total_visitors"] = df["enter_num"] + df["exit_num"]
df["ex_enter_diff"] =  df["exit_num"] - df["enter_num"] 

In [117]:
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,station_name,total_space,lat,lng,day_of_week,...,mrt_station_id,distance,mrt_station_name,mrt_station,enter_num,exit_num,month,bike_rate,total_visitors,ex_enter_diff
0,500101001,2024-04-16,22,7.833333,20.166667,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,3,...,BR08,0.015941,科技大樓,科技大樓,564.0,361.0,4.0,0.279762,925.0,-203.0
1,500101001,2024-04-16,23,23.000000,5.000000,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,3,...,BR08,0.015941,科技大樓,科技大樓,194.0,120.0,4.0,0.821429,314.0,-74.0
2,500101001,2024-04-17,0,20.500000,7.500000,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,...,BR08,0.015941,科技大樓,科技大樓,69.0,43.0,4.0,0.732143,112.0,-26.0
3,500101001,2024-04-17,1,16.666667,11.333333,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,...,BR08,0.015941,科技大樓,NaN,NaN,NaN,NaN,0.595238,NaN,NaN
4,500101001,2024-04-17,2,18.000000,10.000000,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,...,BR08,0.015941,科技大樓,NaN,NaN,NaN,NaN,0.642857,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438546,500119091,2024-04-29,16,11.500000,6.500000,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,...,BR08,0.890423,科技大樓,科技大樓,816.0,749.0,4.0,0.638889,1565.0,-67.0
438547,500119091,2024-04-29,17,9.000000,9.000000,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,...,BR08,0.890423,科技大樓,科技大樓,1442.0,1152.0,4.0,0.500000,2594.0,-290.0
438548,500119091,2024-04-29,18,0.500000,17.500000,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,...,BR08,0.890423,科技大樓,科技大樓,2042.0,1650.0,4.0,0.027778,3692.0,-392.0
438549,500119091,2024-04-29,19,6.666667,11.333333,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,...,BR08,0.890423,科技大樓,科技大樓,1294.0,898.0,4.0,0.370370,2192.0,-396.0


In [118]:
import math

In [119]:
def logit_transform(x):
    if x==0:
        x = 10**(-3)
    elif x>=1:
        x=0.9999
    return(math.log(x/(1-x)))
def square_root_transform(x):
    if x>=1:
        x=1
    return (x**2)
def log_transform(x):
    if x>=1:
        x=1
    elif x==0:
        x = 10**(-3)
    return(math.log(x))
def inverse_transform(x):
    if x>=1:
        x=1
    elif x==0:
        x = 10**(-3)
    return(1/x)

In [120]:
df["bike_rate"] = df["bike_rate"].apply(logit_transform)
# df["bike_rate"] = df["bike_rate"].apply(square_root_transform)
# df["bike_rate"] = df["bike_rate"].apply(log_transform)
# df["bike_rate"] = df["bike_rate"].apply(inverse_transform)

In [121]:
df = df.loc[~df["hour"].isin([22,23,0,1,2,3,4,5]),]

In [122]:
df.loc[df["mrt_station"].isna(),"mrt_station_id"].unique()

array(['O12'], dtype=object)

In [123]:
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,station_name,total_space,lat,lng,day_of_week,...,mrt_station_id,distance,mrt_station_name,mrt_station,enter_num,exit_num,month,bike_rate,total_visitors,ex_enter_diff
8,500101001,2024-04-17,6,23.500000,4.500000,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,...,BR08,0.015941,科技大樓,科技大樓,152.0,193.0,4.0,1.652923,345.0,41.0
9,500101001,2024-04-17,7,17.833333,10.166667,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,...,BR08,0.015941,科技大樓,科技大樓,741.0,938.0,4.0,0.561955,1679.0,197.0
10,500101001,2024-04-17,8,3.166667,24.833333,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,...,BR08,0.015941,科技大樓,科技大樓,1774.0,2219.0,4.0,-2.059507,3993.0,445.0
11,500101001,2024-04-17,9,0.666667,27.333333,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,...,BR08,0.015941,科技大樓,科技大樓,982.0,1402.0,4.0,-3.713572,2384.0,420.0
12,500101001,2024-04-17,10,0.500000,27.500000,YouBike2.0_捷運科技大樓站,28,25.02605,121.54360,4,...,BR08,0.015941,科技大樓,科技大樓,470.0,678.0,4.0,-4.007333,1148.0,208.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438546,500119091,2024-04-29,16,11.500000,6.500000,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,...,BR08,0.890423,科技大樓,科技大樓,816.0,749.0,4.0,0.570545,1565.0,-67.0
438547,500119091,2024-04-29,17,9.000000,9.000000,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,...,BR08,0.890423,科技大樓,科技大樓,1442.0,1152.0,4.0,0.000000,2594.0,-290.0
438548,500119091,2024-04-29,18,0.500000,17.500000,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,...,BR08,0.890423,科技大樓,科技大樓,2042.0,1650.0,4.0,-3.555348,3692.0,-392.0
438549,500119091,2024-04-29,19,6.666667,11.333333,YouBike2.0_臺大明達館北側(員工宿舍),18,25.01816,121.54469,2,...,BR08,0.890423,科技大樓,科技大樓,1294.0,898.0,4.0,-0.530628,2192.0,-396.0


In [124]:
bk_ids = list(df["bike_station_id"].unique())
corr_df = pd.DataFrame(columns=["bike_station_id",
                                "corr_enter_bike",
                                "corr_exit_bike",
                                "corr_total_visitors",
                                "corr_ex_enter_diff",
                                "corr_enter_exit"])

In [125]:
for bk_id in bk_ids:
    tem = df.loc[df["bike_station_id"]==bk_id,]
    cor_matrix = tem.loc[:,["bike_rate","enter_num","total_visitors","ex_enter_diff","exit_num"]].corr()
    corr_df_tem = pd.DataFrame({
        "bike_station_id":[bk_id],
        "corr_enter_bike":[cor_matrix.loc["enter_num","bike_rate"]],
        "corr_exit_bike":cor_matrix.loc["exit_num","bike_rate"],
        "corr_total_visitors":cor_matrix.loc["total_visitors","bike_rate"],
        "corr_ex_enter_diff":cor_matrix.loc["ex_enter_diff","bike_rate"],
        "corr_enter_exit":cor_matrix.loc["exit_num","enter_num"]
    })
    corr_df = pd.concat([corr_df,corr_df_tem],axis=0)

C:\Users\andy\AppData\Local\Temp\ipykernel_14384\3527425555.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  corr_df = pd.concat([corr_df,corr_df_tem],axis=0)


In [126]:
corr_df.sort_values(["corr_ex_enter_diff"],ascending=True,inplace=True)
corr_df

,bike_station_id,corr_enter_bike,corr_exit_bike,corr_total_visitors,corr_ex_enter_diff,corr_enter_exit
0,500119048,0.569666,-0.056100,0.347806,-0.738163,0.578933
0,500113040,0.090182,-0.503709,-0.264032,-0.681918,0.583025
0,500105005,0.332880,-0.609534,-0.150617,-0.636143,-0.046347
0,500101181,0.434360,-0.120876,0.228895,-0.629833,0.578933
0,500110026,0.513630,-0.167003,0.256376,-0.579524,0.256228
...,...,...,...,...,...,...
0,500109065,NaN,NaN,NaN,NaN,0.828852
0,500110028,NaN,NaN,NaN,NaN,0.034303
0,500111081,NaN,NaN,NaN,NaN,0.749373
0,500111085,NaN,NaN,NaN,NaN,0.665863


In [127]:
corr_df.loc[corr_df["corr_ex_enter_diff"]<=-0.5,]

,bike_station_id,corr_enter_bike,corr_exit_bike,corr_total_visitors,corr_ex_enter_diff,corr_enter_exit
0,500119048,0.569666,-0.056100,0.347806,-0.738163,0.578933
0,500113040,0.090182,-0.503709,-0.264032,-0.681918,0.583025
0,500105005,0.332880,-0.609534,-0.150617,-0.636143,-0.046347
0,500101181,0.434360,-0.120876,0.228895,-0.629833,0.578933
0,500110026,0.513630,-0.167003,0.256376,-0.579524,0.256228
0,500101009,0.498822,0.062870,0.356607,-0.550293,0.578933
0,500119045,0.483725,0.042914,0.337424,-0.549125,0.578933
0,500110076,0.517346,-0.116006,0.288159,-0.544989,0.256228
0,500101163,0.431688,-0.211051,0.124677,-0.539576,0.306073
0,500101111,0.231571,-0.064605,0.088175,-0.530415,0.842839


In [128]:
corr_df.loc[corr_df["corr_ex_enter_diff"]>0.5,]

,bike_station_id,corr_enter_bike,corr_exit_bike,corr_total_visitors,corr_ex_enter_diff,corr_enter_exit
0,500110007,-0.330347,0.222943,-0.044671,0.500037,0.406511
0,500101091,-0.416314,-0.164874,-0.307876,0.502083,0.851051
0,500110088,-0.409508,0.266331,-0.052993,0.506498,0.154108
0,500106097,-0.339929,0.314770,0.011150,0.507741,0.177717
0,500101168,-0.442370,0.111100,-0.241980,0.515000,0.354560
0,500106080,-0.150219,0.149320,0.003321,0.521645,0.836240
0,500101210,-0.504173,0.094287,-0.228266,0.524531,0.394115
0,500108015,-0.474507,0.198733,-0.154647,0.531709,0.232940
0,500106006,-0.219892,0.084295,-0.071868,0.544262,0.842839
0,500106048,-0.519248,-0.047822,-0.329786,0.544597,0.587696


In [129]:
corr_df.loc[corr_df["bike_station_id"]==500105005,]

,bike_station_id,corr_enter_bike,corr_exit_bike,corr_total_visitors,corr_ex_enter_diff,corr_enter_exit
0,500105005,0.33288,-0.609534,-0.150617,-0.636143,-0.046347


In [130]:
corr_df_merge = corr_df.merge(mrt_station_table,how="left",on="bike_station_id")

In [131]:
corr_df_merge 

,bike_station_id,corr_enter_bike,corr_exit_bike,corr_total_visitors,corr_ex_enter_diff,corr_enter_exit,mrt_station_id,distance,mrt_station_name
0,500119048,0.569666,-0.056100,0.347806,-0.738163,0.578933,G07,0.076323,公館
1,500113040,0.090182,-0.503709,-0.264032,-0.681918,0.583025,BL10,0.095685,龍山寺
2,500105005,0.332880,-0.609534,-0.150617,-0.636143,-0.046347,G06,0.102163,萬隆
3,500101181,0.434360,-0.120876,0.228895,-0.629833,0.578933,G07,0.102041,公館
4,500110026,0.513630,-0.167003,0.256376,-0.579524,0.256228,BL16,0.400525,忠孝敦化
...,...,...,...,...,...,...,...,...,...
1408,500109065,NaN,NaN,NaN,NaN,0.828852,R21,0.411349,奇岩
1409,500110028,NaN,NaN,NaN,NaN,0.034303,G17,0.481150,台北小巨蛋
1410,500111081,NaN,NaN,NaN,NaN,0.749373,BL22,1.289267,南港
1411,500111085,NaN,NaN,NaN,NaN,0.665863,BR24,0.218697,南港展覽館


In [132]:
df_res = df.merge(corr_df_merge.loc[:,["bike_station_id",
                              "corr_ex_enter_diff",
                              ]],
                              on="bike_station_id",
                              how="left")

In [133]:
def if_cor_greather_than_05(x):
    if x<=-0.5:
        return("負相關")
    elif x>=0.5:
        return("正相關")
    else:
        return("線性相關低")
df_res["if_high"] = df_res["corr_ex_enter_diff"].apply(if_cor_greather_than_05)

In [134]:
df_res.to_csv("correlation_scatter_plot_all_logit.csv",index=False,encoding="utf-8-sig")

In [135]:
bike_corr_high_id = list(corr_df.loc[(corr_df["corr_ex_enter_diff"]>=0.5)|(corr_df["corr_ex_enter_diff"]<=-0.5),"bike_station_id"])
bike_corr_high_id

[500119048,
 500113040,
 500105005,
 500101181,
 500110026,
 500101009,
 500119045,
 500110076,
 500101163,
 500101111,
 500101021,
 500106004,
 500103048,
 500119070,
 500110007,
 500101091,
 500110088,
 500106097,
 500101168,
 500106080,
 500101210,
 500108015,
 500106006,
 500106048,
 500101194,
 500106107,
 500101025,
 500108150,
 500101034]

In [136]:
df_for_scatter_plot = df.loc[df["bike_station_id"].isin(bike_corr_high_id),]

In [137]:
df_for_scatter_plot = df_for_scatter_plot.merge(corr_df_merge.loc[:,["bike_station_id",
                              "corr_ex_enter_diff",
                              ]],
                              on="bike_station_id",
                              how="left")

In [138]:
def if_cor_greather_than_05(x):
    if x<=-0.5:
        return("負相關")
    elif x>=0.5:
        return("正相關")
    else:
        return("線性相關低")
df_for_scatter_plot["if_high"] = df_for_scatter_plot["corr_ex_enter_diff"].apply(if_cor_greather_than_05)

In [143]:
df_for_scatter_plot.to_csv("df_high_low_cor_logit.csv",index=False,encoding="utf-8-sig")

In [142]:
df_for_scatter_plot

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,station_name,total_space,lat,lng,day_of_week,...,mrt_station_name,mrt_station,enter_num,exit_num,month,bike_rate,total_visitors,ex_enter_diff,corr_ex_enter_diff,if_high
0,500101009,2024-04-17,6,0.000000,16.000000,YouBike2.0_新生南路三段66號前,16,25.01976,121.53384,4,...,公館,公館,290.0,361.0,4.0,-6.906755,651.0,71.0,-0.550293,負相關
1,500101009,2024-04-17,7,0.166667,15.833333,YouBike2.0_新生南路三段66號前,16,25.01976,121.53384,4,...,公館,公館,941.0,1866.0,4.0,-4.553877,2807.0,925.0,-0.550293,負相關
2,500101009,2024-04-17,8,0.166667,15.833333,YouBike2.0_新生南路三段66號前,16,25.01976,121.53384,4,...,公館,公館,1678.0,2996.0,4.0,-4.553877,4674.0,1318.0,-0.550293,負相關
3,500101009,2024-04-17,9,0.000000,16.000000,YouBike2.0_新生南路三段66號前,16,25.01976,121.53384,4,...,公館,公館,1108.0,2078.0,4.0,-6.906755,3186.0,970.0,-0.550293,負相關
4,500101009,2024-04-17,10,0.000000,16.000000,YouBike2.0_新生南路三段66號前,16,25.01976,121.53384,4,...,公館,公館,804.0,1469.0,4.0,-6.906755,2273.0,665.0,-0.550293,負相關
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5998,500119070,2024-04-29,16,8.000000,40.000000,YouBike2.0_臺大椰林小舖,48,25.01561,121.53583,2,...,公館,公館,1747.0,1286.0,4.0,-1.609438,3033.0,-461.0,-0.506643,負相關
5999,500119070,2024-04-29,17,13.333333,34.666667,YouBike2.0_臺大椰林小舖,48,25.01561,121.53583,2,...,公館,公館,2994.0,2172.0,4.0,-0.955511,5166.0,-822.0,-0.506643,負相關
6000,500119070,2024-04-29,18,4.833333,43.166667,YouBike2.0_臺大椰林小舖,48,25.01561,121.53583,2,...,公館,公館,3326.0,2169.0,4.0,-2.189532,5495.0,-1157.0,-0.506643,負相關
6001,500119070,2024-04-29,19,2.166667,45.833333,YouBike2.0_臺大椰林小舖,48,25.01561,121.53583,2,...,公館,公館,2234.0,1660.0,4.0,-3.051822,3894.0,-574.0,-0.506643,負相關


In [20]:
df.to_csv("test2.csv",encoding="utf-8-sig",index=False)

In [21]:
#############################